In [2]:
import re
import os
if not 'id_0123456789876543210' in locals():
    _rootlevel = 1
    _oldwd = re.sub(r'\\', '/', os.getcwd())
    _spdirs = _oldwd.split('/')
    _newwd = '/'.join(_spdirs[:(len(_spdirs)-_rootlevel)])
    os.chdir(_newwd)
    id_0123456789876543210 = None

In [3]:
from src.python.dataframe import Catalogue
from src.python.dataframe import filter
from src.python.util import save_object
from src.python.util import load_object
from src.python.util import save_json
from src.python.util import load_json
import pandas as pd
import numpy as np
from os import path


# Lectura y limpieza de datos

### Catálogo



In [3]:
catalogue = Catalogue()

SI_NO = {'1': 'SI', '2': 'NO', '97': 'NA'}
entidad = load_json(path.join('data', 'covid', 'dictionary', 'entidad.json'))

catalogue.add(column='EDAD', name='edad',
              function=lambda x: [int(x) for x in x])
catalogue.add(column='SEXO', name='sexo',
              category={'1': 'MUJER', '2': 'HOMBRE'})
catalogue.add(column='EMBARAZO', name='embarazo',
              category=SI_NO)
catalogue.add(column='NACIONALIDAD', name='nacionalidad',
              category={'1': 'MEXICANA', '2': 'EXTRANGERA'})
catalogue.add(column='MIGRANTE', name='migrante',
              category=SI_NO)
catalogue.add(column='INDIGENA', name='indigena',
              category=SI_NO)
catalogue.add(column='HABLA_LENGUA_INDIG', name='lengua_indigena',
              category=SI_NO)
catalogue.add(column='DIABETES', name='diabetes',
              category=SI_NO)
catalogue.add(column='EPOC', name='epoc',
              category=SI_NO)
catalogue.add(column='ASMA', name='asma',
              category=SI_NO)
catalogue.add(column='INMUSUPR', name='inmunosupresion',
              category=SI_NO)
catalogue.add(column='HIPERTENSION', name='hipertension',
              category=SI_NO)
catalogue.add(column='CARDIOVASCULAR', name='cardiovascular',
              category=SI_NO)
catalogue.add(column='OBESIDAD', name='obesidad',
              category=SI_NO)
catalogue.add(column='RENAL_CRONICA', name='renal_cronica',
              category=SI_NO)
catalogue.add(column='TABAQUISMO', name='tabaquismo',
              category=SI_NO)
catalogue.add(column='OTRA_COM', name='otra_comorbilidad',
              category=SI_NO)
# Variables regresoras
catalogue.add(column='TIPO_PACIENTE', name='tipo',
              category={'1': 'AMBULATORIO', '2': 'HOSPITALIZADO'})
catalogue.add(column='INTUBADO', name='intubado',
              category=SI_NO)
catalogue.add(column='NEUMONIA', name='neumonia',
              category=SI_NO)
catalogue.add(column='UCI', name='uci',
              category=SI_NO)
# Clasificación
catalogue.add(column='OTRO_CASO', name='contacto_covid',
              category=SI_NO)
catalogue.add(column='RESULTADO_LAB', name='laboratorio',
              category={'1': 'POSITIVO', '2': 'NEGATIVO', '3': 'PENDIENTE', '4': 'NO VALIDO', '97': 'NA'})
catalogue.add(column='RESULTADO_ANTIGENO', name='antigeno',
              category={'1': 'POSITIVO', '2': 'NEGATIVO', '97': 'NA'})
catalogue.add(column='CLASIFICACION_FINAL', name='clasificacion',
              category={'1': 'ASOCIACION', '2': 'DICTAMINACION', '3': 'CONFIRMACION',
                        '4': 'NO VALIDO', '5': 'PENDIENTE', '6': 'SOSPECHOSO', '7': 'NEGATIVO'})
# Fechas
catalogue.add(column='FECHA_INGRESO', name='fecha_ingreso',
              function=lambda x: pd.to_datetime(x, format='%Y-%m-%d'))
catalogue.add(column='FECHA_SINTOMAS', name='fecha_sintomas',
              function=lambda x: pd.to_datetime(x, format='%Y-%m-%d'))
catalogue.add(column='FECHA_DEF', name='fecha_defuncion',
              default=pd.NaT,
              exception='9999-99-99',
              function=lambda x: pd.to_datetime(x, format='%Y-%m-%d'))
# Otros
catalogue.add(column='ORIGEN', name='origen',
              category={'1': 'USMER', '2': 'FUERA DE USMER'})
catalogue.add(column='SECTOR', name='sector',
              category={'1': 'CRUZ ROJA', '2': 'DIF', '3': 'ESTATAL', '4': 'IMSS', '5': 'IMSS-BIENESTAR',
                        '6': 'ISSSTE', '7': 'MUNICIPAL', '8': 'PEMEX', '9': 'PRIVADA', '10': 'SEDENA',
                        '11': 'SEMAR', '12': 'SSA', '13': 'UNIVERSITARIO'})
catalogue.add(column='ENTIDAD_UM', name='entidad',
              category=entidad)
catalogue.add(column='ENTIDAD_NAC', name='entidad_nacimiento',
              category=entidad)
catalogue.add(column='ENTIDAD_RES', name='entidad_residencia',
              category=entidad)
catalogue.add(column='MUNICIPIO_RES', name='municipio_residencia',
              function=lambda x: x)
catalogue.add(column='PAIS_NACIONALIDAD', name='pais_nacionalidad',
              function=lambda x: ['NE' if (x == 'SE DESCONOCE' or x == '99')
                                  else x.upper() for x in x])
catalogue.add(column='PAIS_ORIGEN', name='pais_origen',
              function=lambda x: ['NE' if x == 'SE DESCONOCE'
                                  else ('MÉXICO' if x == '97' else x.upper()) for x in x])


### Lectura

In [4]:
def read_history(file):
    return pd.read_csv(path.join('data', 'covid', 'history', file), dtype=str)


# Juntas bases históricas
superdata = read_history('COVID19MEXICO2024.csv')
for subdata in [read_history(f'COVID19MEXICO202{i}.csv') for i in [3, 2, 1, 0]]:
    superdata = pd.concat([superdata,
                           subdata[~subdata.ID_REGISTRO.isin(superdata.ID_REGISTRO)]])
del subdata

# Guardar contadores iniciales
save_json({col: superdata[col].value_counts().to_dict()
           for col in superdata.columns if col != 'ID_REGISTRO'},
          path.join('data', 'covid', 'history', 'initial_counts.json'))

superdata = (filter(superdata, catalogue)
             .sort_values('fecha_ingreso').reset_index(drop=True))

# Guardar contadores finales
save_json({col: {str(k): v for k, v in superdata[col].value_counts().to_dict().items()}
           for col in superdata.columns},
          path.join('data', 'covid', 'history', 'final_counts.json'))

del catalogue

### Clasificación

In [10]:
for name, value in {'indeterminados': ['NO VALIDO', 'PENDIENTE', 'SOSPECHOSO'],
                    'positivos': ['ASOCIACION', 'DICTAMINACION', 'CONFIRMACION'],
                    'negativos': ['NEGATIVO']
                    }.items():
    locs = superdata.clasificacion.isin(value)
    save_object(superdata[locs],
                path.join('data', 'covid', 'cleanned', f'{name}.pkl'))
    superdata = superdata[~locs]
    print(f'{name[0].upper()}{name[1:]}: {sum(locs)}')

del superdata, locs, name, value

{'indeterminados': 1, 'positivos': 1, 'negativos': 1}

# Creación de datos de entrenamiento

### Variables de entrenamiento

In [50]:
# edad máxima 100
# días máximos 16

data = load_object(path.join('data', 'covid', 'cleanned', 'positivos.pkl'))
data['sexo'] = data.sexo + '_' + data.embarazo
data['nacionalidad'] = data.nacionalidad + '_' + data.migrante
data['indigena'] = data.indigena + '_' + data.lengua_indigena
data['dias'] = [x.days for x
                in data.fecha_ingreso-data.fecha_sintomas]

data['hospitalizacion'] = data.tipo == 'HOSPITALIZADO'
data['defuncion'] = ~pd.isna(data.fecha_defuncion)

fecha_etapa = np.array([data.fecha_ingreso.min()] +
                       [pd.Timestamp(x) for x in
                        ['2020-09-20', '2021-05-16', '2021-11-21',
                         '2022-04-17', '2022-10-23', '2023-06-25']])
etapa = pd.Series('G', data.index)
for i, n in enumerate(['A', 'B', 'C', 'D', 'E', 'F']):
    etapa[(data.fecha_ingreso >= fecha_etapa[i])
          & (data.fecha_ingreso < fecha_etapa[i+1])] = n
data['etapa'] = etapa

catalogue = Catalogue()
catalogue.add('dias', function=lambda x: [
              max(0.0, min(1.0, x / 16.0)) for x in x])
catalogue.add('edad', function=lambda x: [
              max(0.0, min(1.0, x / 100.0)) for x in x])
catalogue.add('sexo', category={
    'MUJER_NO': 'MUJER',
    'HOMBRE_NA': 'HOMBRE',
    'MUJER_SI': 'MUJER_SI_EMBARAZO',
    'MUJER_NE': 'MUJER_NE_EMBARAZO'
})
catalogue.add('nacionalidad', category={
    'MEXICANA_NE': 'MEXICANA',
    'MEXICANA_NO': 'MEXICANA',
    'MEXICANA_SI': 'MEXICANA',
    'EXTRANGERA_NO': 'EXTRANGERA',
    'EXTRANGERA_SI': 'EXTRANGERA_SI_MIGRANTE',
    'EXTRANGERA_NE': 'EXTRANGERA_NE_MIGRANTE'
})
catalogue.add('indigena', category={
    'NO_NO': 'NO',
    'SI_SI': 'SI',
    'SI_NO': 'SI',
    'NO_SI': 'SI',
    'SI_NE': 'SI',
    'NE_SI': 'SI'
})
for column in ['diabetes', 'epoc', 'asma', 'inmunosupresion',
               'hipertension', 'cardiovascular', 'obesidad',
               'renal_cronica', 'tabaquismo', 'otra_comorbilidad']:
    catalogue.add(column)
catalogue.add('etapa')
catalogue.add('hospitalizacion')
catalogue.add('defuncion')

save_object(filter(data, catalogue),
            path.join('data', 'covid', 'classification', 'positivos.pkl'))

del data, catalogue, column

### Estandarización de datos de datos

In [51]:
data = load_object(
    path.join('data', 'covid', 'classification', 'positivos.pkl'))

# Estandarización
data = pd.get_dummies(data,
                      columns=['sexo', 'nacionalidad', 'indigena', 'diabetes', 'epoc', 'asma',
                               'inmunosupresion', 'hipertension', 'cardiovascular', 'obesidad',
                               'renal_cronica', 'tabaquismo', 'otra_comorbilidad', 'etapa'])
data = data[data.columns.sort_values()]

# Creación de datos de test (15%  para datos de pruebas)
p_test = 15
n_test = int(len(data) * p_test / 100)
test_index = np.array([True] * n_test + [False] * (len(data) - n_test))
np.random.RandomState(555).shuffle(test_index)

# Guardar datos
save_object(data.drop(['hospitalizacion', 'defuncion'], axis=1)[test_index],
            path.join('data', 'covid', 'classification', 'training', 'xtest.pkl'))
save_object(data[['hospitalizacion', 'defuncion']][test_index],
            path.join('data', 'covid', 'classification', 'training', 'ytest.pkl'))
save_object(data.drop(['hospitalizacion', 'defuncion'], axis=1)[~test_index],
            path.join('data', 'covid', 'classification', 'training', 'xtrain.pkl'))
save_object(data[['hospitalizacion', 'defuncion']][~test_index],
            path.join('data', 'covid', 'classification', 'training', 'ytrain.pkl'))

del data